In [8]:
class Datagram:
    def __init__(self, source, destination):
        self.source = source
        self.destination = destination

In [10]:
class Queue:
    def __init__(self, size):
        self.queue = []
        self.size = size

    def is_empty(self):
        return len(self.queue) == 0
    
    def enqueue(self, datagram):
        if len(self.queue) < self.size:
            self.queue.append(datagram)
            return True
        return False

    def dequeue(self):
        if not self.isEmpty():
            return self.queue.pop(0)
        return None

In [11]:
class Event:
    def __init__(self, instant, destination, datagram):
        self.instant = instant
        self.destination = destination
        self.datagram = datagram

In [12]:
class Router: #roteador
    
    def __init__(self, address, size):
        self.address = address
        self.adjacents = {}
        self.size = size
        self.queue = Queue(self.size)

    #método mapear
    def link(self, adjacent, delay):
        '''
        Ele deve ter o mesmo retorno do método mapear da TabelaDeRepasse: true se o
        mapeamento foi feito ou false caso o endereço já tenha um mapeamento ou a tabela esteja cheia.
        '''
        if adjacent not in self.adjacents:
            self.adjacents[adjacent] = delay
            return True
        else:
            return False


    def receive(self, datagram):
        if datagram.destination != self.address:
            
            if self.queue.size == self.size:
                print(f'Fila em {self.address} estourou ')
            else:
                self.queue.queue(datagram)
            
        else:
            del datagram

    def process(self, instant, action, acao:Router):
        if not self.queue.is_empty():
            datagram = self.queue.dequeue()
            if datagram.destination == self.address:
                del datagram
                return 
            else: 
                    return Event(instant + self.adjacents[acao], acao, datagram)
                    
        else:
            return 

In [13]:
class Network:
    def __init__(self, size):
        self.size = size
        self.quantity = 0
        self.routers = {}

    def add_router(self, router: Router):
        if self.quantity < self.size:
            if router.address not in self.routers:
                self.routers[router.address] = router
                self.quantity += 1
                return True
        return False

In [14]:
class Scheduler:
    def __init__(self, inicial_instant, network, size):
        self.instant = inicial_instant
        self.network = network
        self.size = size
        self.events = []
    
    def schedule(self, instant, router, datagram):
        if len(self.events) == self.size:
            return False
        self.events.append(Event(instant, router, datagram)) #aqui não deveria ser: Event(instant, datagram.source, datagram)?
        return True
    
    def process(self, action):
        for event in self.events:
            if event.instant == self.instant:
                event.destination.receive(event.datagram)
        self.events = [event for event in self.events if event.instant != self.instant]
        for router in self.network.routers:
            router.routing(action[router.address])
            event = router.process(self.instant)
            if event is not None:
                self.events.append(event)
        self.instant += 1


In [ ]:
class Environment:
    def __init__(self, network, scheduler):
        self.network = network
        self.scheduler = scheduler
    
    def get_state(self):
        events_state = []
        for event in self.scheduler.events:
            event_state = (event.instant, event.destination, event.datagram.destination)
            events_state.append(event_state)

        routers_state = []
        for router in self.network.routers:
            router_state = []
            datagrams_state = []
            for datagram in router.datagrams:
                datagrams_state.append(datagram.destination)
            router_state.append(datagrams_state)
            for adjacent in router.adjacents:
                queue_state = []
                for datagram in router.adjacents[adjacent][0].queue:
                    queue_state.append(datagram.destination)
                router_state.append(queue_state)
            routers_state.append(router_state)
            
        return (events_state, routers_state)
    
    def take_action(self, action):
        reward = 0
        for router in self.network.routers:
            reward -= len(router.datagrams)
        self.scheduler.process(action)
        new_state = self.get_state()
        return reward, new_state

obs: Se não for ter criação de evento AGENDADO, o método schedule() parece pouco útil, podemos inserir um novo datagrama diretamente em datagrams do router de origem 

Onde está deletando os eventos?

Ordem dos elementos no estados não deve importar ex: [1, 2] = [2, 1]